In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pandas as pd
from pathlib import Path
import dateutil.parser
from NPw_aux import read_iono_data, process_ionosphere_files, read_EQ_data, process_eq
from datetime import timedelta

In [2]:
p = Path("/home/carloscano/data/EQ/").glob("*.csv")
all_pd = pd.DataFrame()

valid_colums = ["time", "longitude", "latitude", "depth", "mag", "type"]
for file in p:
    print(file)
    all_pd = pd.concat([all_pd, pd.read_csv(file,  parse_dates=['time'], date_parser= dateutil.parser.parse).loc[:, valid_colums]])
all_pd = all_pd[all_pd['type'] == "earthquake"].drop("type", axis = 1).set_index("time").sort_index()
all_pd.to_csv("2016_2020_EQ.csv")

/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [3]:

df_eq = read_EQ_data("/home/carloscano/data/EQ/")

/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [4]:
df_process = process_eq(df_eq,df_ion.columns, 5, 2000, 1000, 4.5, 1.5)

NameError: name 'df_ion' is not defined

In [ ]:
len(df_process)

In [ ]:
df_process

In [ ]:
from NPw_aux import prepare_ion_data

In [ ]:
df_ion, df_eq = prepare_ion_data("GRK")

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from aux_function_SR import read_data, get_eq_filtered, SR_SENSORS
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level, save, load
import logging
import pandas as pd
from datetime import datetime, timedelta
import pickle
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast
from dataclasses import dataclass, asdict
from dateutil.relativedelta import *
from NPw_aux import prepare_ion_data, process_ionosphere_files, remove_outliers
import ast

import sys

In [2]:
set_log_level("ERROR")

delta = timedelta(minutes=30)
# Read SR and EQ data
freq = timedelta(minutes = 60)
df_ion, df_eq = prepare_ion_data("GRK", freq)
remove_outliers(df_ion, 3)

/home/carloscano/data/EQ/
/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [457]:
df_regressor = df_ion.reset_index().copy()
df_events = df_eq.copy()
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast

forecast_length = timedelta(hours=48)
question_mark_length = timedelta(hours=48)
# Time to take into account to predict 
historic_lenght =  timedelta(days=5)
training_lenght = timedelta(days=365)
config_npw_d = {
    "forecast_length": forecast_length,
    "question_mark_length": question_mark_length,
    "freq": freq,
    "training_length": training_lenght,
    "historic_lenght": historic_lenght,
    "drop_missing": False,
    "num_hidden_layers": 10,
    "learning_rate": None,
    "d_hidden": 16,
    "yearly_seasonality": True,
    "daily_seasonality" : True,
    "weekly_seasonality" : True,
    "seasonal_mode": "multiplicative",
    "seasonal_reg": 0,
    "multivariate_season": True,
    "multivariate_trend": True,
    "verbose": False,
    "epochs": 50,
    "drop_missing": False,
    "use_gpu": False,
    "event_type": "Non-Binary",
    "event_mode": "multiplicative",
    "normalization": True,
    "trend_regularization": 0,
    "trend_n_changepoint": 10,
    "sparce_AR": None,
    "loss_func": "Huber",
    "growth": "off"
}

config_npw = ConfigNPw(**config_npw_d)
ConfigEQ_d = {
    "dist_start": 1000,
    "dist_delta": 3000,
    "mag_start": 4.5,
    "mag_delta": 2,
    "filter": True,
    "drop": ["arc_cos", "arc_sin"]
}
config_events = ConfigEQ(**ConfigEQ_d)
from NPw_aux import prepare_eq
_, _, df_events_filtered = prepare_eq(df_events,  config_events.dist_start,
    config_events.dist_delta,
    config_events.mag_start,
    config_events.mag_delta,
    config_events.filter,
    None)
print(len(df_events_filtered))
synthetic_events = pd.read_pickle("synthetic.pkl")
#config_path = "config_002.json"
#(config_npw, config_events) = NPw.save_config(config_path)

m1: 4.5 m2: 6.5 d1: 1000 d2: 4000
1636


In [458]:
a = "a"
if a: 
    print("HELLO")

HELLO


In [465]:
NPw_o = NPw(config_npw, df_regressor, config_events, df_events, synthetic_events)

m1: 4.5 m2: 6.5 d1: 1000 d2: 4000


In [466]:
hours_offsets = [0]
event_offsets = [None,  -timedelta(hours=24)]
index = 1


# see what happen in 2019-02-01T10:00:00 Nan values at the end
start_day = datetime.fromisoformat("2018-10-01T10:00:00")

start_date = start_day + index * relativedelta(months=+1)

In [467]:

from scipy import stats

In [472]:
number_of_folds = 4*365
NPw_o.get_folds(k = number_of_folds)

In [ ]:
for n_fold in range(number_of_folds):
    NPw_o.process_fold(n_fold)

Finding best initial lr:   0%|          | 0/260 [00:00<?, ?it/s]

In [316]:
import pyarrow.feather as feather
feather.write_feather(NPw_o.get_results(), 'test')


WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/pyarrow/feather.py:156: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = Table.from_pandas(df, preserve_index=preserve_index)



In [94]:
8 //3

2

In [470]:
df, df_MAE = NPw_o.get_results()

In [471]:
df

BASE    SYNT_0                                          \
                        0   28800.0   57600.0   86400.0  115200.0  144000.0   
2018-12-30 duth  5.005888  4.936865  4.975390  4.942408  4.958411  4.997257   
           larm  4.737707  4.666318  4.703883  4.671105  4.688290  4.730739   
           noa1  3.914851  3.839358  3.881329  3.853726  3.860299  3.912227   
2020-12-30 duth  8.025856  7.992598  7.979591  8.035449  8.032310  8.020372   
           noa1  8.037347  8.001587  7.988757  8.045821  8.043661  8.029375   

                   SYNT_1                                ...    SYNT_4  \
                  28800.0   57600.0   86400.0  115200.0  ...   28800.0   
2018-12-30 duth  4.973962  4.990266  4.976304  4.983072  ...  4.915769   
           larm  4.704148  4.720037  4.706168  4.713433  ...  4.645712   
           noa1  3.879318  3.897045  3.885374  3.888148  ...  3.817698   
2020-12-30 duth  8.010689  8.004968  8.029571  8.028186  ...  8.020560   
           noa1  8.021016  8.015375  8.040506  8.039553  ...  8.031642   

                                                           SYNT_5            \
                  57600.0   86400.0  115200.0  144000.0   28800.0   57600.0   
2018-12-30 duth  4.969412  4.923497  4.945791  4.999788  4.902721  4.967171   
           larm  4.698024  4.652387  4.676330  4.735322  4.633508  4.696354   
           noa1  3.876182  3.837747  3.846909  3.919078  3.804944  3.875228   
2020-12-30 duth  8.018600  8.027039  8.026564  8.024756  8.044579  8.051232   
           noa1  8.029708  8.038329  8.038001  8.035839  8.057550  8.064109   

                                               
                  86400.0  115200.0  144000.0  
2018-12-30 duth  4.912015  4.938809  5.003603  
           larm  4.641535  4.670309  4.741077  
           noa1  3.829063  3.840076  3.926642  
2020-12-30 duth  8.022545  8.024165  8.030318  
           noa1  8.034813  8.035928  8.043289  

[5 rows x 31 columns]

In [391]:
df_scale = df.apply(lambda x: ((x -x[0])/x[0]), axis = 1)

In [405]:
df_scale = ((df_scale / df_scale.abs().max().max()) *100).astype(int)

In [406]:
NPw.print_df(df_scale)

In [344]:
df_scale.to_csv("binary_ex.csv")

In [412]:
print(type(df.index))
b = list()
[b.append(item[0]) for item in df.index if item not in b]
b = (pd.DataFrame(b, columns = ["dates"]).drop_duplicates())
b

<class 'pandas.core.indexes.multi.MultiIndex'>


,dates
0,2017-05-24 04:00:00
3,2017-10-17 08:00:00
6,2018-03-12 12:00:00
9,2018-08-05 16:00:00
12,2018-12-29 20:00:00
15,2019-05-25 00:00:00
18,2019-10-18 04:00:00
21,2020-03-12 08:00:00
24,2020-08-05 12:00:00
26,2020-12-29 16:00:00


In [438]:
def filter_events(df_events, fc_dt, sort_value, low_td, high_td):
    df_events.index = df_events.index + config_npw.forecast_length // 2
    events = df_events.iloc[(df_events.index > fc_dt - low_td) & (df_events.index < fc_dt + high_td)].sort_values("pr", ascending = False).head()
    if events.empty == False:
        return events

In [439]:
current_events = list()
for fc_dt in b["dates"]:
    current_events.append(filter_events(df_events.copy(), fc_dt, "pr", timedelta(0), timedelta(150)))

In [442]:
synthetic_events = [x.iloc[0] for x in current_events[2:8]]

In [447]:
pd.concat(synthetic_events,axis = 1, ignore_index=True).T.to_pickle("synthetic2.pkl")

In [62]:
df = feather.read_feather('test').T
df

2017-10-17 09:00:00                     2018-08-05 18:00:00  \
                               duth      larm      noa1                duth   
BASE   0                   8.814728  9.767968  9.383814            8.974570   
SYNT92 28800.0             8.814925  9.767439  9.382515            9.008780   
       57600.0             8.827728  9.781843  9.398255            9.023551   
       86400.0             8.836466  9.790316  9.407853            9.015770   
       115200.0            8.837470  9.791109  9.407445            9.006664   
       144000.0            8.810431  9.763589  9.380588            9.015599   
SYNT27 28800.0             8.815568  9.767602  9.382197            8.971719   
       57600.0             8.836795  9.791484  9.408293            8.970333   
       86400.0             8.851269  9.805520  9.424192            8.971063   
       115200.0            8.852931  9.806833  9.423516            8.971918   
       144000.0            8.808114  9.761216  9.379001            8.971079   
SYNT45 28800.0             8.814692  9.767793  9.383486            8.981127   
       57600.0             8.817165  9.770575  9.386526            8.984209   
       86400.0             8.818855  9.772213  9.388382            8.982585   
       115200.0            8.819049  9.772366  9.388303            8.980686   
       144000.0            8.813825  9.767050  9.383114            8.982550   
SYNT79 28800.0             8.815598  9.767614  9.382194            8.956538   
       57600.0             8.837115  9.791823  9.408647            8.947195   
       86400.0             8.851786  9.806051  9.424763            8.952120   
       115200.0            8.853471  9.807382  9.424077            8.957875   
       144000.0            8.808041  9.761141  9.378954            8.952228   
SYNT46 28800.0             8.818552  9.769485  9.383068            8.978362   
       57600.0             8.858896  9.814882  9.432670            8.980162   
       86400.0             8.886356  9.841516  9.462830            8.979214   
       115200.0            8.889507  9.844007  9.461548            8.978105   
       144000.0            8.804363  9.757330  9.376984            8.979193   
SYNT68 28800.0             8.814695  9.767634  9.383154            8.964332   
       57600.0             8.820020  9.773624  9.389699            8.959204   
       86400.0             8.823656  9.777150  9.393694            8.961907   
       115200.0            8.824074  9.777480  9.393524            8.965066   
       144000.0            8.812828  9.766034  9.382353            8.961966   

                                    2019-05-25 03:00:00                      \
                     larm      noa1                duth      larm      noa1   
BASE   0         9.653240  8.783305            3.156135  2.558178  2.993283   
SYNT92 28800.0   9.689484  8.818835            3.157778  2.564330  2.997336   
       57600.0   9.705843  8.832165            3.144294  2.548145  2.984061   
       86400.0   9.696770  8.823076            3.149658  2.551067  2.986776   
       115200.0  9.687281  8.816473            3.147894  2.549713  2.984670   
       144000.0  9.699158  8.824235            3.139362  2.539924  2.977418   
SYNT27 28800.0   9.650216  8.780337            3.156287  2.558983  2.993800   
       57600.0   9.648680  8.779085            3.154192  2.556466  2.991736   
       86400.0   9.649532  8.779939            3.155025  2.556920  2.992158   
       115200.0  9.650422  8.780558            3.154751  2.556710  2.991831   
       144000.0  9.649308  8.779830            3.153428  2.555193  2.990706   
SYNT45 28800.0   9.660193  8.790126            3.156641  2.560513  2.994796   
       57600.0   9.663607  8.792908            3.150909  2.553629  2.989150   
       86400.0   9.661713  8.791011            3.153187  2.554870  2.990304   
       115200.0  9.659734  8.789633            3.152438  2.554295  2.989409   
       144000.0  9.662211  8.791252            3.148816  2.550142

In [64]:
from scipy.stats import zscore
zetascore_table=zscore(df,axis=0)
zetascore_table

2017-10-17 09:00:00                     2018-08-05 18:00:00  \
                               duth      larm      noa1                duth   
BASE   0                  -0.678100 -0.669662 -0.682251           -0.124629   
SYNT92 28800.0            -0.668954 -0.693488 -0.738624            1.661598   
       57600.0            -0.074032 -0.044912 -0.055620            2.432888   
       86400.0             0.331960  0.336561  0.360869            2.026587   
       115200.0            0.378601  0.372265  0.343152            1.551117   
       144000.0           -0.877749 -0.866795 -0.822224            2.017664   
SYNT27 28800.0            -0.639043 -0.686150 -0.752400           -0.273491   
       57600.0             0.347241  0.389135  0.379949           -0.345894   
       86400.0             1.019770  1.021102  1.069834           -0.307738   
       115200.0            1.097009  1.080232  1.040495           -0.263131   
       144000.0           -0.985416 -0.973655 -0.891085           -0.306900   
SYNT45 28800.0            -0.679740 -0.677544 -0.696462            0.217742   
       57600.0            -0.564831 -0.552279 -0.564546            0.378673   
       86400.0            -0.486335 -0.478533 -0.484018            0.293871   
       115200.0           -0.477316 -0.471628 -0.487445            0.194707   
       144000.0           -0.720036 -0.710991 -0.712597            0.292012   
SYNT79 28800.0            -0.637673 -0.685571 -0.752544           -1.066165   
       57600.0             0.362105  0.404428  0.395300           -1.554043   
       86400.0             1.043818  1.045028  1.094601           -1.296874   
       115200.0            1.122107  1.104962  1.064862           -0.996387   
       144000.0           -0.988792 -0.977015 -0.893130           -1.291231   
SYNT46 28800.0            -0.500384 -0.601333 -0.714627            0.073379   
       57600.0             1.374175  1.442646  1.437713            0.167358   
       86400.0             2.650095  2.641856  2.746408            0.117834   
       115200.0            2.796510  2.753978  2.690797            0.059929   
       144000.0           -1.159673 -1.148604 -0.978606            0.116749   
SYNT68 28800.0            -0.679608 -0.684685 -0.710875           -0.659205   
       57600.0            -0.432214 -0.414990 -0.426860           -0.926949   
       86400.0            -0.263263 -0.256255 -0.253535           -0.785833   
       115200.0           -0.243850 -0.241394 -0.260910           -0.620901   
       144000.0           -0.766384 -0.756711 -0.745619           -0.782736   

                                    2019-05-25 03:00:00                      \
                     larm      noa1                duth      larm      noa1   
BASE   0        -0.123378 -0.125627            0.032037 -0.024305  0.005079   
SYNT92 28800.0   1.632691  1.728344            0.121417  0.283709  0.237014   
       57600.0   2.425332  2.423942           -0.612133 -0.526559 -0.522696   
       86400.0   1.985704  1.949681           -0.320361 -0.380236 -0.367295   
       115200.0  1.525960  1.605133           -0.416287 -0.448063 -0.487837   
       144000.0  2.101439  2.010139           -0.880478 -0.938104 -0.902851   
SYNT27 28800.0  -0.269912 -0.280495            0.040295  0.016015  0.034655   
       57600.0  -0.344315 -0.345803           -0.073672 -0.109966 -0.083449   
       86400.0  -0.303031 -0.301264           -0.028389 -0.087266 -0.059328   
       115200.0 -0.259903 -0.268938           -0.043285 -0.097790 -0.078039   
       144000.0 -0.313896 -0.306939           -0.115260 -0.173693 -0.142384   
SYNT45 28800.0   0.213509  0.230309            0.059560  0.092619  0.091662   
       57600.0   0.378896  0.375467           -0.252280 -0.252021 -0.231442   
       86400.0   0.287136  0.276479           -0.128335 -0.189877 -0.165417   
       115200.0  0.191256  0.204615           -0.169098 -0.218688 -0.216632   
       144000.0  0.311284  0.289094           -0.366165 -0.426588

In [60]:
dates = [x[0] for x in df.keys()].unique()

AttributeError: 'list' object has no attribute 'unique'

In [59]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(50))]

,depth,mag,dist,arc_cos,arc_sin,pr
dates,,,,,,
2018-06-16 19:00:00,59.18,2.90,9700.010747,0.225588,0.974223,0.010867
2018-06-16 20:00:00,10.00,4.90,6155.838436,0.982205,0.187814,0.119868
2018-06-16 21:00:00,10.00,4.90,15889.900259,0.999717,0.023771,0.046437
2018-06-16 22:00:00,10.00,3.20,4889.963970,0.992138,0.125148,0.028864
2018-06-16 23:00:00,0.70,3.60,8796.323781,0.079021,0.996873,0.023680
...,...,...,...,...,...,...
2020-12-31 19:00:00,7.98,3.60,10270.632955,0.687918,0.725788,0.020281
2020-12-31 20:00:00,10.00,4.30,6451.777623,0.468243,0.883600,0.063794
2020-12-31 21:00:00,5.00,3.34,8774.055019,0.963969,0.266014,0.018434


In [32]:
df

2017-10-17 09:00:00                       2018-08-05 18:00:00  \
                     duth       larm       noa1                duth   
count           31.000000  31.000000  31.000000           31.000000   
mean             8.829322   9.782841   9.399537            8.976957   
std              0.021877   0.022577   0.023427            0.019468   
min              8.804363   9.757330   9.376984            8.947195   
25%              8.814711   9.767608   9.382791            8.963149   
50%              8.819049   9.772366   9.388382            8.974570   
75%              8.837292   9.791653   9.408470            8.981838   
max              8.889507   9.844007   9.462830            9.023551   

                            2019-05-25 03:00:00                        \
            larm       noa1                duth       larm       noa1   
count  31.000000  31.000000           31.000000  31.000000  31.000000   
mean    9.655787   8.785712            3.155546   2.558663   2.993194   
std     0.020980   0.019481            0.018686   0.020306   0.017763   
min     9.623750   8.756073            3.116470   2.516167   2.956245   
25%     9.641112   8.771904            3.148355   2.549927   2.985500   
50%     9.653240   8.783305            3.153187   2.554295   2.990304   
75%     9.660953   8.790568            3.156726   2.559748   2.994298   
max     9.705843   8.832165            3.212638   2.622739   3.048050   

      2020-03-12 12:00:00                       2020-12-29 21:00:00             
                     duth       larm       noa1                duth       noa1  
count           31.000000  31.000000  31.000000           31.000000  31.000000  
mean             0.980014   0.997003   0.951975            9.021997   9.160137  
std              0.036887   0.043812   0.039492            0.009737   0.009864  
min              0.925915   0.941938   0.891083            8.996952   9.134714  
25%              0.964322   0.974183   0.929412            9.019873   9.157673  
50%              0.973868   0.984455   0.951547            9.025393   9.163607  
75%              0.990199   1.006304   0.962089            9.026577   9.165054  
max              1.084895   1.119578   1.064704            9.037532   9.176082

In [ ]:
df_events.iloc[(df_events.index > df.keys()[1][0] - timedelta(0)) & (df_events.index < df.keys()[1][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [51]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(0)) & (df_events.index < df.keys()[4][0] + timedelta(2))].sort_values("pr", ascending = False).head()

,depth,mag,dist,arc_cos,arc_sin,pr
dates,,,,,,


In [49]:
df_events.iloc[(df_events.index > df.keys()[6][0] - timedelta(0)) & (df_events.index < df.keys()[6][0] + timedelta(2))].sort_values("pr", ascending = False).head()

,depth,mag,dist,arc_cos,arc_sin,pr
dates,,,,,,
2019-05-24 21:00:00,10.00,4.3,302.474252,0.763268,0.646083,1.360729
2019-05-26 07:00:00,122.57,8.0,11192.117635,0.999704,0.024332,1.345796
2019-05-25 22:00:00,27.54,4.2,285.379409,0.981383,0.192063,1.308527
2019-05-26 12:00:00,10.00,4.5,458.418421,0.832288,0.554344,1.090707
2019-05-26 23:00:00,6.51,4.1,664.142739,0.990726,0.135879,0.510140


In [50]:
df_events.iloc[(df_events.index > df.keys()[10][0] - timedelta(0)) & (df_events.index < df.keys()[10][0] + timedelta(2))].sort_values("pr", ascending = False).head()

,depth,mag,dist,arc_cos,arc_sin,pr
dates,,,,,,
2020-03-12 15:00:00,34.68,4.3,366.838606,0.148671,0.988887,1.121980
2020-03-13 07:00:00,10.00,4.9,2253.343174,0.976176,0.216981,0.327463
2020-03-13 18:00:00,10.00,4.9,5154.785223,0.353746,0.935342,0.143146
2020-03-12 17:00:00,10.00,5.3,9102.046254,0.739088,0.673609,0.119638
2020-03-12 22:00:00,10.00,5.3,9430.772522,0.966650,0.256102,0.115468


In [176]:
len(NPw_o.list_keys_fold)

6

In [182]:
(NPw_o.list_test_RMSE[0].reset_index())

WARNING - (py.warnings._showwarnmsg) - <ipython-input-182-dd2755884f80>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  (NPw_o.list_test_RMSE[0].reset_index())



index      BASE               SYNT23                       \
                0 08/29/2017, 16:00:00 08/30/2017, 04:00:00   
0  duth  3.175303             3.184243             3.174414   
1  larm  6.781397             6.784064             6.779957   
2  noa1  3.254088             3.265502             3.259054   

                                                                  \
  08/30/2017, 16:00:00 08/31/2017, 04:00:00 08/31/2017, 15:00:00   
0             3.180692             3.165026             3.171870   
1             6.784463             6.776524             6.776911   
2             3.264258             3.251451             3.248363   

                SYNT21                                            ...  \
  08/29/2017, 16:00:00 08/30/2017, 04:00:00 08/30/2017, 16:00:00  ...   
0             3.203187             3.179412             3.194609  ...   
1             6.790953             6.780988             6.791922  ...   
2             3.288787             3.273222             3.285788  ...   

                SYNT56                                            \
  08/29/2017, 16:00:00 08/30/2017, 04:00:00 08/30/2017, 16:00:00   
0             3.183104             3.174336             3.179936   
1             6.783689             6.780027             6.784046   
2             3.264072             3.258320             3.262962   

                                                          SYNT42  \
  08/31/2017, 04:00:00 08/31/2017, 15:00:00 08/29/2017, 16:00:00   
0             3.165964             3.172067             3.227113   
1             6.776965             6.777310             6.800558   
2             3.251538             3.248784             3.317548   

                                                                  \
  08/30/2017, 04:00:00 08/30/2017, 16:00:00 08/31/2017, 04:00:00   
0             3.190587             3.213952             3.155411   
1             6.785181             6.802053             6.772305   
2             3.293683             3.312954             3.265374   

                        
  08/31/2017, 15:00:00  
0             3.181084  
1             6.773758  
2             3.253828  

[3 rows x 32 columns]

In [52]:
a = NPw_o.list_test_RMSE[0][0:3].copy()
b = NPw_o.list_test_RMSE[0][3:6].copy()
c = NPw_o.list_test_RMSE[0][3:6].copy()
a.co"]
b.columns = ["Blumns = ["A"]
c.columns = ["C"]
print(a)
print(b)
r1 = pd.concat([a,b, c], axis = 1).copy()
r2 = pd.concat([a,b, c], axis = 1).copy()
r3 = pd.concat([a,b, c], axis = 1).copy()
print(pd.concat([r1,r2,a], axis=1, keys = [1,2,"NOTHING"]))

             A
duth  3.088523
larm  2.736721
noa1  2.148144
             B
duth  3.085403
larm  2.732932
noa1  2.145125
             1                             2                       NOTHING
             A         B         C         A         B         C         A
duth  3.088523  3.085403  3.085403  3.088523  3.085403  3.085403  3.088523
larm  2.736721  2.732932  2.732932  2.736721  2.732932  2.732932  2.736721
noa1  2.148144  2.145125  2.145125  2.148144  2.145125  2.145125  2.148144


In [ ]:
from sdv.tabular import GaussianCopula

In [76]:
model = GaussianCopula()
model.fit(events)

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column 'depth'. Data will not be rounded.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column 'mag'. Data will not be rounded.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column 'dist'. Data will not be rounded.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column 'arc_cos'. Data will not be rounded.
  warni

In [102]:
sample = model.sample(2000)
sample.head()
print(sample.nlargest(1, columns = ["mag"]))
print(sample.nlargest(5, columns = ["dist"]))
example = sample.nlargest(5, columns = ["dist"])
idx = example.index[[True, False, True, False, True]]

        depth       mag    dist   arc_cos   arc_sin        pr
504  0.990824  0.685528  0.7123  0.991131  0.049847  0.151101
         depth       mag      dist   arc_cos   arc_sin        pr
703   0.980156  0.009138  0.999971  0.795092  0.552872  0.283331
1831  0.999560  0.011413  0.999952  0.851509  0.419824  0.181082
1253  0.982143  0.016596  0.999817  0.979923  0.082546  0.245447
1692  0.995353  0.018816  0.999807  0.696397  0.685709  0.191014
695   0.966928  0.026510  0.999789  0.657546  0.701471  0.319161


In [124]:
for idx, ex in example.iterrows():
    print(ex)
    for e in ex.index:
        print(e)

depth      5.000000
mag        0.009138
dist       0.999971
arc_cos    0.795092
arc_sin    0.552872
pr         0.283331
Name: 703, dtype: float64
depth
mag
dist
arc_cos
arc_sin
pr
depth      0.999560
mag        0.011413
dist       0.999952
arc_cos    0.851509
arc_sin    0.419824
pr         0.181082
Name: 1831, dtype: float64
depth
mag
dist
arc_cos
arc_sin
pr
depth      5.000000
mag        0.016596
dist       0.999817
arc_cos    0.979923
arc_sin    0.082546
pr         0.245447
Name: 1253, dtype: float64
depth
mag
dist
arc_cos
arc_sin
pr
depth      0.995353
mag        0.018816
dist       0.999807
arc_cos    0.696397
arc_sin    0.685709
pr         0.191014
Name: 1692, dtype: float64
depth
mag
dist
arc_cos
arc_sin
pr
depth      5.000000
mag        0.026510
dist       0.999789
arc_cos    0.657546
arc_sin    0.701471
pr         0.319161
Name: 695, dtype: float64
depth
mag
dist
arc_cos
arc_sin
pr


In [146]:
len(df_train)

26277

In [145]:
df_train = NPw_o.df_train
delta_t = NPw_o.config_npw.forecast_length

In [50]:
t0 = df_train.iloc[-(NPw_o.n_forecasts)]["ds"]
t1 = t0 + delta_t

In [51]:
series = pd.date_range(start=t0, end=t1, periods=5)
series.values

array(['2018-12-28T00:00:00.000000000', '2018-12-28T12:00:00.000000000',
       '2018-12-29T00:00:00.000000000', '2018-12-29T12:00:00.000000000',
       '2018-12-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [52]:
df_test = df_train.copy()

In [42]:
df_test["ds"].get_loc()

AttributeError: 'Series' object has no attribute 'get_loc'

In [149]:
print(ev.iloc[1])
print(ev.iloc[2])
ev.iloc[2] = ev.iloc[1]
print(ev.iloc[2])

depth      0.115096
mag        0.736503
dist       0.166585
arc_cos    0.787443
arc_sin    0.787443
pr         0.308815
Name: 37, dtype: float64
depth      0.115096
mag        0.736503
dist       0.166585
arc_cos    0.787443
arc_sin    0.787443
pr         0.308815
Name: 85, dtype: float64
depth      0.115096
mag        0.736503
dist       0.166585
arc_cos    0.787443
arc_sin    0.787443
pr         0.308815
Name: 85, dtype: float64


In [141]:
for index, (df_train, df_test) in enumerate(NPw_o.folds):
    if index == 10:
        print(df_test)


                      ds          y    ID
0    2019-02-19 18:00:00  19.734861  duth
1    2019-02-19 21:00:00  20.727333  duth
2    2019-02-20 00:00:00  20.738965  duth
3    2019-02-20 03:00:00  20.750598  duth
4    2019-02-20 06:00:00  20.762230  duth
...                  ...        ...   ...
4081 2019-08-08 09:00:00  24.958944  noa1
4082 2019-08-08 12:00:00  24.307972  noa1
4083 2019-08-08 15:00:00  24.940583  noa1
4084 2019-08-08 18:00:00  24.649556  noa1
4085 2019-08-08 21:00:00  19.739083  noa1

[4086 rows x 3 columns]


In [87]:
config_fc = ConfigForecast(
    start_forecast=start_date, offset_event=None
)
metrics= NPw_o.add_forecast(config_fc
)

2018-11-01 09:00:00
2018-10-30 09:00:00
                      ds          y    ID     depth       mag      dist  \
0    2017-11-01 12:00:00  25.725944  duth  0.000000  0.000000  0.000000   
1    2017-11-01 15:00:00  21.906417  duth  0.000000  0.000000  0.000000   
2    2017-11-01 18:00:00  20.057917  duth  0.000000  0.000000  0.000000   
3    2017-11-01 21:00:00  19.974917  duth  0.000000  0.000000  0.000000   
4    2017-11-02 00:00:00  20.154750  duth  0.000000  0.000000  0.000000   
...                  ...        ...   ...       ...       ...       ...   
9030 2017-10-11 21:00:00        NaN   NaN  0.025204  0.295455  0.002895   
9031 2017-10-14 09:00:00        NaN   NaN  0.021841  0.159091  0.019350   
9032 2017-10-23 00:00:00        NaN   NaN  0.031516  0.159091  0.032260   
9033 2017-10-24 09:00:00        NaN   NaN  0.021841  0.181818  0.035727   
9034 2017-10-29 06:00:00        NaN   NaN  0.049120  0.136364  0.028213   

       arc_cos   arc_sin        pr  
0     0.000000  0.0000

Finding best initial lr:   0%|          | 0/248 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

...

2018-11-01 09:00:00
      current                              pred                      
         duth       larm       noa1        duth       larm       noa1
0   25.530583  26.139306  25.153056  120.661316  30.584166  29.049374
1   24.821472  25.887722  25.218333  121.930656  30.873896  29.317024
2   21.021333  22.491556  21.754306    117.0709  29.764643  28.292316
3   20.926722  21.666611  21.131028  103.877068  26.753111  25.510319
4   19.224528  20.116389  19.245444   93.671204  24.423594  23.358356
5    7.181389   7.543333   7.291528   89.216438  23.406778  22.419044
6    7.835222   8.318778   8.560611   94.945381  24.714426  23.627022
7    9.835222  10.220639  10.398528  120.706322   30.59444  29.058865
8   11.149944  10.894250  11.092556   130.42453  32.812649  31.108006
9   11.607806  11.705278  12.284167   131.82518  33.132347  31.403339
10   7.529500   8.015222   8.493889  127.044052  32.041046  30.395214
11   7.153000   7.043500   7.642556  113.856667  29.030985  27.614578


Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/neuralprophet/plot_model_parameters_plotly.py:373: RuntimeWarning:

invalid value encountered in divide




In [80]:
metrics[1]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> Trend <i style="color:#fc9944">~9h</i>',
              'type': 'scatter',
              'uid': '86e14430-d90e-4ed1-817a-91574773541c',
              'x': array([datetime.datetime(2017, 11, 1, 12, 0),
                          datetime.datetime(2017, 11, 1, 18, 0),
                          datetime.datetime(2017, 11, 1, 21, 0), ...,
                          datetime.datetime(2018, 10, 31, 18, 0),
                          datetime.datetime(2018, 10, 31, 21, 0),
                          datetime.datetime(2018, 11, 1, 6, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([46.43117191, 46.31756029, 46.26075448, ..., 65.7386202 , 65.7112081 ,
                          65.62897178]),
              'yaxis': 'y'},
             {'marker': {'color': '#2d92ff'},
             

In [ ]:
NPw_o.one_step_test(model, config_fc, df_test)

In [277]:
future = model.one_step(df_test, n_historic_predictions=24)

In [ ]:
forecast = model.predict(future, decompose=False, raw=True)
forecast_2  = forecast[forecast["ds"] == start_date].T

In [335]:
forecast_2.columns = df_test_2.columns  + "_pred"
df_test_2.columns =  df_test_2.columns  + "_current"

In [343]:
forecast_2 = forecast_2.drop(["ds" ,"ID"]).reset_index().drop("index", axis = 1)

In [414]:
df_test

,ds,y,ID,depth,mag,dist,arc_cos,arc_sin,pr
0,2019-03-27 10:00:00,24.289000,duth,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-03-27 11:00:00,24.899583,duth,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-27 12:00:00,23.751583,duth,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-03-27 13:00:00,23.437917,duth,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-03-27 14:00:00,23.762417,duth,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
427,2019-04-02 05:00:00,8.873250,noa1,0.0,0.0,0.0,0.0,0.0,0.0
428,2019-04-02 06:00:00,9.546583,noa1,0.0,0.0,0.0,0.0,0.0,0.0
429,2019-04-02 07:00:00,10.731167,noa1,0.0,0.0,0.0,0.0,0.0,0.0
430,2019-04-02 08:00:00,11.372333,noa1,0.0,0.0,0.0,0.0,0.0,0.0


In [416]:
pd.pivot(df_test, index="ds", columns="ID", values="y")[-24:]

ID,duth,larm,noa1
ds,,,
2019-04-01 10:00:00,24.909083,26.871500,27.418417
2019-04-01 11:00:00,24.880333,26.595500,26.971833
2019-04-01 12:00:00,25.879583,27.841750,28.300500
2019-04-01 13:00:00,24.773750,26.981000,26.955500
2019-04-01 14:00:00,23.035500,25.411333,24.747583
2019-04-01 15:00:00,21.813167,24.049583,23.176583
2019-04-01 16:00:00,21.707667,23.461250,22.821000
2019-04-01 17:00:00,21.727500,23.359750,22.887667
2019-04-01 18:00:00,21.717500,23.232750,23.120083


In [374]:
df_all = pd.concat([df_test_2.reset_index(), forecast_2], axis = 1).drop("ds", axis = 1)
df_all.columns = df_all.columns.str.split('_', expand=True)
df_all = df_all.swaplevel(axis=1) 
mse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0)
mae = df_all['current'].sub(df_all['pred']).abs().mean(axis=0)
rmse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0).pow(1/2)
results = pd.concat([mse, mae, rmse], axis = 1)
results

,0,1,2
duth,74.581824,6.296380,8.636077
larm,67.118254,6.268352,8.192573
noa1,51.606940,6.086715,7.183797


In [409]:
 pd.pivot(df_test, index="ds", columns="ID", values="y")[-(self.n_forecasts):]

KeyError: 'ds'

In [293]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix

df_test_2.apply(mean_squared_error, axis=0)


TypeError: mean_squared_error() missing 1 required positional argument: 'y_pred'